In [7]:
from pymongo import MongoClient
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import networkx as nx
import sys
from sympy import degree
from tqdm import tqdm
import math
import seaborn as sns
import powerlaw as plw
import datetime
import h5py
from IPython.display import display, clear_output
import plotly.graph_objects as go
from reddit import plotting, network
from reddit import datahandling as dh
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
def add_data(db_from, db_to, collection_from, collection_to, fields_selection , subreddit, db_address='127.0.0.1:27017', ):

    """
    Copies selected fields from a database collection to another.
    Args:
        db_from (str): origin database
        db_to (str): target database
        collection_from (str): origin collection
        collection_to (str): target collection
        fields_selection (str): Set of fields to copy. Valid options: 'comments','submissions'
    Returns:
        None
    """

    global fields
    if fields_selection == 'comments':
        fields = ['author', 'created_utc', 'id', 'subreddit', 'score', 'link_id', 'parent_id']
    elif fields_selection == 'submissions':
        fields = ['author', 'created_utc', 'id', 'subreddit', 'score', 'domain', 'num_comments']
    else:
        raise ValueError('field_selection must be "comments" or "submissions"')
    client = MongoClient('mongodb://' + db_address)
    project = {field:1 for field in fields}
    project['_id'] = 0

    pipeline = [
    {'$project': project},
    {'$match': {'subreddit': subreddit}},
    {'$merge':{'into':{'db': db_to, 'coll': collection_to}}}
    ]
    client[db_from][collection_from].aggregate(pipeline)

In [3]:
db_from = 'reddit'
db_to = 'reddit_subreddits'
collection_from = 'comments'
collection_to = 'politics'
add_data(db_from, db_to, collection_from, collection_to, 'comments', subreddit='politics')


In [11]:
db_from = 'reddit'
db_to = 'reddit_subreddits'
collection_from = 'comments'
collection_to = 'AskReddit'
add_data(db_from, db_to, collection_from, collection_to, 'comments', subreddit='AskReddit')


In [10]:
client = MongoClient('localhost', 27017)
db = client.reddit

In [11]:
subreddits = ['AskScienceDiscussion', 'AskMen', 'AskScience', 'AskWomen', 'CompSci', 'DesMoines', 'IAmA', 'MachineLearning', 'Movies', 'MyLittlePony', 'PersonalFinance', 'TalesFromTechSupport', 'WashingtonDC']
for sr in subreddits:
    dh.save_to_hdf5(db, 'comments', 'data/raw/{}.h5'.format(sr), sr)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [12]:
subreddits = ['askscience', 'compsci', 'desmoines', 'movies', 'mylittlepony', 'personalfinance', 'talesfromtechsupport', 'washingtondc']
for sr in subreddits:
    dh.save_to_hdf5(db, 'comments', 'data/raw/{}.h5'.format(sr), sr)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [6]:
df = pd.read_hdf('data/raw/politics.h5',key = 'df',stop=10000)
print(df)
df = dh.processRaw(df, rem_bots=False, rem_del=False)
print(df)
df[['author','id','parent_id', 'parent_author']] = df[['author','id','parent_id', 'parent_author']].astype(object)
df[['user_id']] = df[['user_id']].astype(int)
#df.to_hdf('data/proc/politics.h5', key='df')

                   author       id   parent_id
0               [deleted]  c696o5c   t3_10002q
1             doctorquien  c6979au  t1_c69788m
2             willanthony  c697bcr  t1_c6979au
3             doctorquien  c697djv  t1_c697bcr
4          DonkeyDickDoak  c696q84   t3_1000ce
...                   ...      ...         ...
9995          puntcuncher  c69fymz   t3_101058
9996                 85IQ  c69gmr7   t3_101058
9997              lazyFer  c69fuou   t3_10105g
9998  YetAnotherRandomGuy  c69g19h   t3_10106o
9999            [deleted]  c69gtn9   t3_10106o

[10000 rows x 3 columns]


  0%|          | 0/10000 [00:00<?, ?it/s]

               author       id parent_id  user_id parent_author
0           [deleted]  c696o5c    10002q     1558           NaN
4      DonkeyDickDoak  c696q84    1000ce      382           NaN
5          CRYMTYPHON  c696srf    1000ce      224           NaN
19    1000000students  c696wf6    1000en        5           NaN
6            IrishJoe  c696ydr    1000ce      673           NaN
...               ...      ...       ...      ...           ...
1750        [deleted]  c6z1boi   c6z07ro     1558           NaN
1751           curien  c6z8gv1   c6z1boi     1890           NaN
1752        [deleted]  c6zbsp6   c6z8gv1     1558           NaN
1753        [deleted]  c79p7m3   c6z8gv1     1558           NaN
9069  iPeeLavaLampGoo  c7qkbij   c69jdun     2265           NaN

[10000 rows x 5 columns]


In [20]:
iterator = pd.read_hdf('data/raw/politics.h5',key = 'df',iterator=True,chunksize=1000000)
for i in tqdm(iterator):
    i = dh.remove_bots(i)
    i = pd.DataFrame(i)
    i['parent_id'] = i['parent_id'].astype('string').str.replace('t3_','')
    i['parent_id'] = i['parent_id'].astype('string').str.replace('t1_','')
    i[['parent_id']] = i[['parent_id']].astype(object)
    i = pd.DataFrame(i)
    i.to_hdf('data/proc/politics_rem_bots_2.h5', key='df', append=True)

0it [00:00, ?it/s]

                author       id   parent_id
0            [deleted]  c696o5c   t3_10002q
1          doctorquien  c6979au  t1_c69788m
2          willanthony  c697bcr  t1_c6979au
3          doctorquien  c697djv  t1_c697bcr
4       DonkeyDickDoak  c696q84   t3_1000ce
...                ...      ...         ...
999995    BrianDawkins  c6xjwrd  t1_c6xjuww
999996       Caelestia  c6xk9av  t1_c6xjuww
999997    BrianDawkins  c6xjw76  t1_c6xjv6a
999998       [deleted]  c6xk1sl  t1_c6xjvhx
999999   Frantic_Child  c6xjxa0  t1_c6xjw76

[987876 rows x 3 columns]


1it [00:09,  9.81s/it]

                     author       id   parent_id
0       Unwanted_Commentary  c6xjxvh  t1_c6xjwrd
1                 [deleted]  c6xk12w  t1_c6xjxa0
2             AdelleChattre  c6xk1mz  t1_c6xjxa0
3                     wwabc  c6xkdri  t1_c6xjxa0
4                   Mookhaz  c6xkerr  t1_c6xjxa0
...                     ...      ...         ...
999995          eaturbrainz  c7s814p  t1_c7s3lgf
999996         peabodygreen  c7s82zb  t1_c7s3lgf
999997   TracyMorganFreeman  c7s3w50  t1_c7s3ll9
999998         Crunkenstien  c7s9ed7  t1_c7s3lu8
999999  Paranoidexboyfriend  c7sbo71  t1_c7s3lu8

[988768 rows x 3 columns]


2it [00:19,  9.93s/it]

...

158it [26:20, 10.08s/it]

                      author       id   parent_id
0                  [deleted]  c518l74  t1_c512tnh
1             charles_drawin  c518qnw  t1_c512tnh
2         whats_ina_username  c518rht  t1_c512tnh
3                     UpDown  c518rqe  t1_c512tnh
4       I_am_not_a_black_guy  c518u1h  t1_c512tnh
...                      ...      ...         ...
999995             qisqisqis  c5tkz4q    t3_y9h3u
999996              sirbruce  c5tl2wv    t3_y9h3u
999997             Menace117  c5tl3kk    t3_y9h3u
999998         glutenfree123  c5tl4x1    t3_y9h3u
999999               Jakabov  c5tl5r6    t3_y9h3u

[988066 rows x 3 columns]


159it [26:30,  9.83s/it]

                 author       id parent_id
0               johny5w  c5tl61a  t3_y9h3u
1             gloomdoom  c5tlb98  t3_y9h3u
2             Rettet181  c5tljze  t3_y9h3u
3            fargosucks  c5tlng9  t3_y9h3u
4          SezitLykItiz  c5tlpkc  t3_y9h3u
...                 ...      ...       ...
641222        Messier77  c696olg  t3_zzzw7
641223  ArkadiusMaximus  c696zp6  t3_zzzw7
641224        Mysticbus  c6975ul  t3_zzzw7
641225        markevens  c69boo4  t3_zzzw7
641226        [deleted]  c69df27  t3_zzzw7

[633386 rows x 3 columns]


160it [26:36,  9.98s/it]


In [6]:
iterator = pd.read_hdf('data/proc/politics_rem_bots_2.h5',columns=['author'], key = 'df',chunksize=10000000)
for n,i in enumerate(iterator):
    sys.getsizeof(i)
print(n)

KeyboardInterrupt: 